<a href="https://colab.research.google.com/github/dondonrocket/kokudo/blob/%E3%83%86%E3%82%B9%E3%83%88/hasegawa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# 0. ライブラリ読み込み
# =========================
import pandas as pd
import numpy as np
import geopandas as gpd
import lightgbm as lgb
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree

In [ ]:
# =========================
# 1. train / test 読み込み
# =========================
train = pd.read_csv(
    "/content/train.csv",
    encoding="shift_jis",
    encoding_errors="replace",
    low_memory=False
)

test = pd.read_csv(
    "/content/test.csv",
    encoding="shift_jis",
    encoding_errors="replace",
    low_memory=False
)

In [ ]:
# =========================
# DID
# =========================

import geopandas as gpd
import pandas as pd

# =========================
# 1. DID shp 読み込み
# =========================
did_gdf = gpd.read_file("/content/A16-20_00_DID.shp")  # アップロードした shp
did_gdf = did_gdf.to_crs(epsg=4326)  # train/test と CRS を合わせる

# DID の識別子列
did_id_col = "A16_001"

# 参考に人口や面積の列を選定
did_pop_col = "A16_005"  # 人口
did_area_col = "A16_006"  # 面積
did_gdf["DID_density"] = did_gdf[did_pop_col] / (did_gdf[did_area_col] + 1e-6)

# =========================
# 2. train/test GeoDataFrame化
# =========================
train_gdf = gpd.GeoDataFrame(
    train,
    geometry=gpd.points_from_xy(train["lon"], train["lat"]),
    crs="EPSG:4326"
)
test_gdf = gpd.GeoDataFrame(
    test,
    geometry=gpd.points_from_xy(test["lon"], test["lat"]),
    crs="EPSG:4326"
)

# =========================
# 3. 空間結合: DID 割り当て
# =========================
train_gdf = gpd.sjoin(train_gdf, did_gdf[[did_id_col, did_pop_col, did_area_col, "DID_density", "geometry"]], how="left", predicate="within")
test_gdf  = gpd.sjoin(test_gdf,  did_gdf[[did_id_col, did_pop_col, did_area_col, "DID_density", "geometry"]], how="left", predicate="within")

# 列名を整える
train_gdf.rename(columns={
    did_pop_col: "DID_population",
    did_area_col: "DID_area"
}, inplace=True)
test_gdf.rename(columns={
    did_pop_col: "DID_population",
    did_area_col: "DID_area"
}, inplace=True)

# =========================
# 4. 欠損処理
# =========================
for col in ["DID_population", "DID_area", "DID_density"]:
    train_gdf[col] = train_gdf[col].fillna(0)
    test_gdf[col]  = test_gdf[col].fillna(0)

# =========================
# 5. train/test に反映
# =========================
train = pd.DataFrame(train_gdf.drop(columns="geometry"))
test  = pd.DataFrame(test_gdf.drop(columns="geometry"))

# 確認
train[["DID_population", "DID_area", "DID_density"]].head()

# =========================
# DID特徴量をtrain/testに結合
# =========================
# 例: DID_population, DID_area, DID_density
# DIDのGeoDataFrameを gpd.read_file("DID.shp") で読み込んでいる前提
# train/testのgeometryはすでにEPSG:4326になっていると仮定

from shapely.geometry import Point

def add_DID_features(df, did_gdf, did_columns=["A16_005","A16_006","A16_010"]):
    """
    df: train/testのDataFrame (geometry列必須)
    did_gdf: DIDのGeoDataFrame (geometry列必須)
    did_columns: 追加するDID列名
    """
    df = df.copy()
    df_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326").to_crs(epsg=3857)
    did_gdf_3857 = did_gdf.to_crs(epsg=3857)

    # DIDに最も近いポリゴンを割り当てる
    # KDTreeで距離最小のインデックスを取得
    did_xy = np.vstack([did_gdf_3857.geometry.centroid.x, did_gdf_3857.geometry.centroid.y]).T
    tree = KDTree(did_xy)

    assigned = []
    for pt in df_gdf.geometry:
        dist, idx = tree.query([[pt.x, pt.y]], k=1)
        assigned.append(idx[0][0])

    assigned_df = did_gdf_3857.iloc[assigned][did_columns].reset_index(drop=True)
    assigned_df.index = df.index
    df[did_columns] = assigned_df

    return df

train = add_DID_features(train, did_gdf)
test  = add_DID_features(test, did_gdf)



In [ ]:
# =========================
# 2. 駅乗降客数（2019年）
# =========================
df = pd.read_csv("S12-24_NumberOfPassengers_utf8.csv")
df_2019 = df[(df["S12_039"]==1)&(df["S12_038"]==1)]
station_2019 = df_2019.groupby("S12_001c", as_index=False).agg(passengers_2019=("S12_041","sum")).rename(columns={"S12_001c":"station_code"})

In [ ]:
# =========================
# 3. 駅ポイント（lon / lat）
# =========================
station_point_gdf = gpd.read_file("/content/S12-24_NumberOfPassengers.geojson")
station_point_gdf["S12_001c"] = station_point_gdf["S12_001c"].astype(str)
station_2019["station_code"] = station_2019["station_code"].astype(str)
stations = station_point_gdf.merge(station_2019, left_on="S12_001c", right_on="station_code", how="left")
stations = stations.to_crs(epsg=3857)
stations["geometry"] = stations.geometry.centroid
stations_gdf = stations[["S12_001c","passengers_2019","geometry"]].copy()
stations_gdf.crs = "EPSG:3857"

In [ ]:
# =========================
# 4. 駅特徴量作成関数
# =========================
def add_station_features(df, stations_gdf, radius=500):
    df = df.copy()
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326").to_crs(epsg=3857)
    joined = gpd.sjoin(gdf, stations_gdf, how="left", predicate="dwithin", distance=radius)
    feat = joined.groupby("building_id", as_index=False).agg(
        **{
            f"station_passengers_{radius}m_sum": ("passengers_2019","sum"),
            f"station_passengers_{radius}m_max": ("passengers_2019","max"),
            f"station_passengers_{radius}m_mean": ("passengers_2019","mean")
        }
    )
    df = df.merge(feat, on="building_id", how="left")
    for col in feat.columns:
        if col!="building_id":
            df[col] = df[col].fillna(0)
            df[col+"_log"] = np.log1p(df[col])
    return df

In [ ]:
# =========================
# 5. train/testに駅特徴量付与
# =========================
for radius in [500,1000]:
    train = add_station_features(train, stations_gdf, radius)
    test  = add_station_features(test, stations_gdf, radius)

In [ ]:
# =========================
# 6. 築年 → 築年数
# =========================
for df in [train,test]:
    df["building_create_date"] = pd.to_numeric(df["building_create_date"], errors="coerce")
    df["age"] = (2023 - df["building_create_date"]).clip(0,100)


In [ ]:
# =========================
# 7. 地価公示データ
# =========================
land_gdf = gpd.read_file("/content/L01-23.geojson").to_crs(epsg=4326)
for df in [train,test]:
    df["geometry"] = gpd.points_from_xy(df["lon"], df["lat"])
train_gdf = gpd.GeoDataFrame(train, geometry="geometry", crs="EPSG:4326")
test_gdf  = gpd.GeoDataFrame(test, geometry="geometry", crs="EPSG:4326")
train_gdf = train_gdf.to_crs(epsg=6668)
test_gdf  = test_gdf.to_crs(epsg=6668)
land_gdf  = land_gdf.to_crs(epsg=6668)

land_xy = np.vstack([land_gdf.geometry.x.values, land_gdf.geometry.y.values]).T
tree = KDTree(land_xy)
land_prices = land_gdf['L01_006'].values

def nearest_land_price_fast(pt, tree, land_prices):
    dist, idx = tree.query([[pt.x, pt.y]], k=1)
    return land_prices[idx[0][0]]

train_gdf['nearest_land_price'] = train_gdf['geometry'].apply(lambda pt: nearest_land_price_fast(pt, tree, land_prices))
test_gdf['nearest_land_price']  = test_gdf['geometry'].apply(lambda pt: nearest_land_price_fast(pt, tree, land_prices))
train['final_land_price'] = train_gdf['nearest_land_price'].values
test['final_land_price']  = test_gdf['nearest_land_price'].values

In [ ]:
# =========================
# 9. 学習データ作成
# =========================
y = np.log1p(train["money_room"])
# 数値カラムのみを使用
features = train.select_dtypes(include=[np.number]).columns.tolist()

# 目的変数は除外
features = [c for c in features if c not in ["money_room", "money_hoshou_company"]]
X = train[features]
X_test = test[features]

In [ ]:

# =========================
# 10. LightGBM 学習
# =========================
model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric="mape",verbose=50)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.208961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13823
[LightGBM] [Info] Number of data points in the train set: 291139, number of used features: 97
[LightGBM] [Info] Start training from score 16.872611


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, n_estimators=2000,
              num_leaves=64, random_state=42, subsample=0.8)

In [ ]:
# =========================
# low帯スケーリング補正
# =========================

y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)

LOW_TH = 9_000_000
LOW_SCALE = 0.95  # ← 後で調整

mask_low = y_pred < LOW_TH
y_pred[mask_low] *= LOW_SCALE


In [ ]:

# =========================
# 予測と低価格帯 APE 計算
# =========================
y_pred_train = np.expm1(model.predict(X))
train["y_pred"] = y_pred_train
train["ape"] = np.abs(train["money_room"] - train["y_pred"]) / train["money_room"]

for band in ["low", "mid", "high"]:
    mask = train["price_band"] == band
    print(f"{band} APE mean:", train.loc[mask, "ape"].mean())

low APE mean: 0.24291240639521078
mid APE mean: 0.13385141835785996
high APE mean: 0.11443921387162534


In [ ]:
# =========================
# 12. 予測・提出
# =========================
pred = np.expm1(model.predict(X_test))
submit = pd.DataFrame({"id": test["id"], "money_room": pred})
submit.to_csv("submit.csv", index=False, header=False)
print("submit.csv を出力しました")

submit.csv を出力しました
